# Operaciones básicas (III)

In [1]:
import numpy as np
import pandas as pd

## Tablas pivote

Siguiendo con las funciones de gestión de índices, pandas incluye la posibilidad de gestionar los mismos como si de una Pivot Table de Excel se tratase, haciendo mucho más sencillo el análisis de información resultante.

In [10]:
peliculas = pd.DataFrame(
            {'Año':[2014, 2014, 2013, 2013, 2001], 
             'Valoración':[6, None, 8.75, None, 8.9],
             'Presupuesto':[160, 250, 100, None, 93],
             'Director':['Gareth Edwards', 'Peter Jackson', 'Martin Scorsese', 'Alfonso Cuarón', 'Peter Jackson'],
             'Título':['Godzilla', 'El Hobbit III', 'El lobo de Wall Street', 'Gravity', 'Lord of the Rings']}
)
peliculas

,Año,Valoración,Presupuesto,Director,Título
0,2014,6.00,160.0,Gareth Edwards,Godzilla
1,2014,NaN,250.0,Peter Jackson,El Hobbit III
2,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street
3,2013,NaN,NaN,Alfonso Cuarón,Gravity
4,2001,8.90,93.0,Peter Jackson,Lord of the Rings


In [3]:
# Filas: Año, Columnas: Director, Valores: Título
peliculas.pivot(columns = 'Director', index = 'Año', values = 'Título')

Director,Alfonso Cuarón,Gareth Edwards,Martin Scorsese,Peter Jackson
Año,,,,
2001,NaN,NaN,NaN,Lord of the Rings
2013,Gravity,NaN,El lobo de Wall Street,NaN
2014,NaN,Godzilla,NaN,El Hobbit III


Si bien esto mismo ya lo podíamos realizar con las operaciones vistas hasta ahora.

In [11]:
# Establecemos el índice a las dos variables sobre las que queremos "pivotar"
p = peliculas.set_index(['Año', 'Director'])
p

Valoración  Presupuesto                  Título
Año  Director                                                        
2014 Gareth Edwards         6.00        160.0                Godzilla
     Peter Jackson           NaN        250.0           El Hobbit III
2013 Martin Scorsese        8.75        100.0  El lobo de Wall Street
     Alfonso Cuarón          NaN          NaN                 Gravity
2001 Peter Jackson          8.90         93.0       Lord of the Rings

In [12]:
# Pasamos el último nivel de índice de filas a columnas
p = p.unstack()
p

Valoración                                               \
Director Alfonso Cuarón Gareth Edwards Martin Scorsese Peter Jackson   
Año                                                                    
2001                NaN            NaN             NaN           8.9   
2013                NaN            NaN            8.75           NaN   
2014                NaN            6.0             NaN           NaN   

            Presupuesto                                               \
Director Alfonso Cuarón Gareth Edwards Martin Scorsese Peter Jackson   
Año                                                                    
2001                NaN            NaN             NaN          93.0   
2013                NaN            NaN           100.0           NaN   
2014                NaN          160.0             NaN         250.0   

                 Título                                         \
Director Alfonso Cuarón Gareth Edwards         Martin Scorsese   
Año                                                              
2001                NaN            NaN                     NaN   
2013            Gravity            NaN  El lobo de Wall Street   
2014                NaN       Godzilla                     NaN   

                             
Director      Peter Jackson  
Año                          
2001      Lord of the Rings  
2013                    NaN  
2014          El Hobbit III

In [13]:
# Elegimos únicamente el valor de la columna Título
p['Título']

Director,Alfonso Cuarón,Gareth Edwards,Martin Scorsese,Peter Jackson
Año,,,,
2001,NaN,NaN,NaN,Lord of the Rings
2013,Gravity,NaN,El lobo de Wall Street,NaN
2014,NaN,Godzilla,NaN,El Hobbit III


También podemos crear tablas pivote utilizando una función de agregación para los valores, de forma que se haga una agrupación de resultados.

In [14]:
peliculas

,Año,Valoración,Presupuesto,Director,Título
0,2014,6.00,160.0,Gareth Edwards,Godzilla
1,2014,NaN,250.0,Peter Jackson,El Hobbit III
2,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street
3,2013,NaN,NaN,Alfonso Cuarón,Gravity
4,2001,8.90,93.0,Peter Jackson,Lord of the Rings


In [20]:
# Hacemos que haya dos películas para el mismo año y director
peliculas.loc[peliculas['Año'] == 2001, 'Año']
peliculas

,Año,Valoración,Presupuesto,Director,Título
0,2014,6.00,160.0,Gareth Edwards,Godzilla
1,2014,NaN,250.0,Peter Jackson,El Hobbit III
2,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street
3,2013,NaN,NaN,Alfonso Cuarón,Gravity
4,2014,8.90,93.0,Peter Jackson,Lord of the Rings


In [21]:
# Hacemos que haya dos películas para el mismo año y director
peliculas.loc[peliculas['Año'] == 2001, 'Año'] = 2014 # Las peliculas del 2001 seleccioname la columna año y cambialas por 2014
peliculas

,Año,Valoración,Presupuesto,Director,Título
0,2014,6.00,160.0,Gareth Edwards,Godzilla
1,2014,NaN,250.0,Peter Jackson,El Hobbit III
2,2013,8.75,100.0,Martin Scorsese,El lobo de Wall Street
3,2013,NaN,NaN,Alfonso Cuarón,Gravity
4,2014,8.90,93.0,Peter Jackson,Lord of the Rings


In [22]:
# Utilizamos la función pivot_table para establecer: valor, índice, columnas y función de agregación en caso de colisión
pd.pivot_table(peliculas, values='Presupuesto', index=['Director'], columns=['Año'], aggfunc=np.sum) #
# Del DataFrame peliculas, selecciona las filas de director, de columna pon año, y muestra sus presupuestos. A los presupuestos los sumas

Año,2013,2014
Director,,
Alfonso Cuarón,0.0,NaN
Gareth Edwards,NaN,160.0
Martin Scorsese,100.0,NaN
Peter Jackson,NaN,343.0


## Eliminación de filas y/o columnas en pandas

Aunque el proceso de eliminación de columnas se puede hacer mediante la aplicación de los mismos métodos que en el caso de diccionarios, pandas pone a nuestra disposición el método <b>drop</b>.

In [23]:
serie = pd.Series([1,2,3,4], index=['a','b','c','d'])
serie

a    1
b    2
c    3
d    4
dtype: int64

In [24]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4), index=['f1', 'f2', 'f3', 'f4'], columns=['c1','c2','c3','c4'])
dataframe

,c1,c2,c3,c4
f1,0,1,2,3
f2,4,5,6,7
f3,8,9,10,11
f4,12,13,14,15


In [25]:
# Eliminación de valores de una Serie
serie.drop('a')

b    2
c    3
d    4
dtype: int64

In [27]:
del serie['b']

KeyError: 'b'

In [28]:
serie

a    1
c    3
d    4
dtype: int64

In [34]:
# Eliminación de filas de un DataFrame
dataframe.drop(['f1',  'f2'])
# De está forma si se eliminaria para siempre
del dataframe['f1']

KeyError: 'f1'

In [33]:
dataframe

,c1,c2,c3,c4
f1,0,1,2,3
f2,4,5,6,7
f3,8,9,10,11
f4,12,13,14,15


In [31]:
# Eliminación de columnas de un DataFrame
dataframe.drop('c2', axis=1)


,c1,c3,c4
f1,0,2,3
f2,4,6,7
f3,8,10,11
f4,12,14,15


## Aritmética con estructuras de pandas

Aunque, como ya se ha visto, podemos aprovechar la compatibilidad con NumPy para llevar a cabo operaciones aritméticas básicas, estas operaciones aplican el proceso de "alineación" de índices introduciendo valores NaN en los resultados cuando no hay coincidencia de claves. Para solucionar este problema, pandas nos ofrece algunas funciones de utilidad para las más básicas (suma, resta, multiplicación y división) que permiten establecer un valor de "relleno" en el caso de claves no coincidentes.

In [35]:
serie1 = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])
serie1

a    1
b    2
c    3
d    4
dtype: int64

In [36]:
serie2 = pd.Series([5, 6, 7, 8], index=['c', 'd', 'e', 'f'])
serie2

c    5
d    6
e    7
f    8
dtype: int64

In [37]:
# Resultado de operación básica"
serie1 + serie2

a     NaN
b     NaN
c     8.0
d    10.0
e     NaN
f     NaN
dtype: float64

In [38]:
# Resultado con operación pandas
serie1.add(serie2)

a     NaN
b     NaN
c     8.0
d    10.0
e     NaN
f     NaN
dtype: float64

In [39]:
# Resultado con operación pandas y relleno
serie1.add(serie2, fill_value=0) # Add tiene el parametro fill_value, entonces suma un valor con uno que no conozca haciendolo pasar por 0

a     1.0
b     2.0
c     8.0
d    10.0
e     7.0
f     8.0
dtype: float64

In [40]:
serie1.sub(serie2, fill_value=0)

a    1.0
b    2.0
c   -2.0
d   -2.0
e   -7.0
f   -8.0
dtype: float64

In [41]:
serie1.mul(serie2, fill_value=0)

a     0.0
b     0.0
c    15.0
d    24.0
e     0.0
f     0.0
dtype: float64

In [44]:
serie1.div(serie2, fill_value=2)

a    0.500000
b    1.000000
c    0.600000
d    0.666667
e    0.285714
f    0.250000
dtype: float64

## Ordenación en estructuras de pandas

pandas pone a nuestra disposición varias formas de realizar ordenaciones de los contenidos de una Serie o un DataFrame. Vamos a ver los más utilizados.

#### Ordenación en Series

In [45]:
serie = pd.Series([3, 2, 1, 4], index=['d', 'a', 'c', 'b'])
serie

d    3
a    2
c    1
b    4
dtype: int64

In [46]:
# Ordenación por índice
serie.sort_index()

a    2
b    4
c    1
d    3
dtype: int64

In [47]:
# Ordenación descendente por índice
serie.sort_index(ascending=False)

d    3
c    1
b    4
a    2
dtype: int64

In [48]:
# Ordenación por valores
serie.sort_values()

c    1
a    2
d    3
b    4
dtype: int64

In [49]:
# Ordenación por valores descendente
serie.sort_values(ascending=False)

b    4
d    3
a    2
c    1
dtype: int64

#### Ordenación en DataFrames

In [50]:
dataframe = pd.DataFrame(np.arange(16).reshape(4, 4), index=['f3', 'f1', 'f4', 'f2'], columns=['c3', 'c1', 'c4', 'c2'])
dataframe

,c3,c1,c4,c2
f3,0,1,2,3
f1,4,5,6,7
f4,8,9,10,11
f2,12,13,14,15


In [51]:
# Ordenación por índice de filas
dataframe.sort_index()

,c3,c1,c4,c2
f1,4,5,6,7
f2,12,13,14,15
f3,0,1,2,3
f4,8,9,10,11


In [52]:
# Ordenación por índice de columnas
dataframe.sort_index(axis=1)

,c1,c2,c3,c4
f3,1,3,0,2
f1,5,7,4,6
f4,9,11,8,10
f2,13,15,12,14


In [53]:
# Ordenación descendente por índice de filas
dataframe.sort_index(ascending=False)

,c3,c1,c4,c2
f4,8,9,10,11
f3,0,1,2,3
f2,12,13,14,15
f1,4,5,6,7


In [54]:
# Ordenación por valores de filas
dataframe.sort_values(['f1'], axis=1)

,c3,c1,c4,c2
f3,0,1,2,3
f1,4,5,6,7
f4,8,9,10,11
f2,12,13,14,15


In [55]:
# Ordenación por valores de columnas
dataframe.sort_values(['c1'])

,c3,c1,c4,c2
f3,0,1,2,3
f1,4,5,6,7
f4,8,9,10,11
f2,12,13,14,15


## Recuperación de muestras parciales del contenido

En estructuras de datos potencialmente grandes, suele ser muy necesaria la recuperación de una muestra de ejemplo de un conjunto reducido de elementos que permitan hacerse una idea del contenido de la estructura sin necesidad de listar TODO el contenido de la misma. Pandas, como R, pone a nuestra disposición dos métodos <b>head</b> (para obtener un muestra del inicio de la estructura) y <b>tail</b> para obtener la muestras del final. Ambos métodos recibirán como parámetro el número de registros a recuperar.

In [56]:
serie = pd.Series(np.arange(100))
serie

0      0
1      1
2      2
3      3
4      4
      ..
95    95
96    96
97    97
98    98
99    99
Length: 100, dtype: int64

In [57]:
dataframe = pd.DataFrame(np.arange(100).reshape(10, 10))
dataframe

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,3,4,5,6,7,8,9
1,10,11,12,13,14,15,16,17,18,19
2,20,21,22,23,24,25,26,27,28,29
3,30,31,32,33,34,35,36,37,38,39
4,40,41,42,43,44,45,46,47,48,49
5,50,51,52,53,54,55,56,57,58,59
6,60,61,62,63,64,65,66,67,68,69
7,70,71,72,73,74,75,76,77,78,79
8,80,81,82,83,84,85,86,87,88,89
9,90,91,92,93,94,95,96,97,98,99


In [63]:
# Recuperación de los 5 primeros elementos de una Serie
serie.head()

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [59]:
# Recueperación de los 5 últimos elementos de una Serie
serie.tail()

95    95
96    96
97    97
98    98
99    99
dtype: int64

In [60]:
# Recuperación de los 3 primeros elementos de un DataFrame
dataframe.head(3)

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,3,4,5,6,7,8,9
1,10,11,12,13,14,15,16,17,18,19
2,20,21,22,23,24,25,26,27,28,29


In [61]:
# Recueperación de los 3 últimos elementos de un DataFrame
dataframe.tail(3)

,0,1,2,3,4,5,6,7,8,9
7,70,71,72,73,74,75,76,77,78,79
8,80,81,82,83,84,85,86,87,88,89
9,90,91,92,93,94,95,96,97,98,99
